### Proceed with data cleaning and some preliminary EDA

In [1]:
import pandas as pd
import numpy as np

In [2]:
trnsact = pd.read_csv('df_trnsact.csv')

In [3]:
trnsact.describe()

,SKU,Store,Register,Trannum,IntID,Quantity,AMT,Orgprice,Duplicate,SEQ,MIC,Unknown
count,1.209169e+08,1.209169e+08,1.209169e+08,1.209169e+08,1.209169e+08,1.209169e+08,1.209169e+08,1.209169e+08,1.209169e+08,1.209169e+08,1.209169e+08,1.209169e+08
mean,4.973132e+06,4.441209e+03,4.004916e+02,3.339482e+03,1.902617e+08,1.000003e+00,3.677663e+01,2.462009e+01,2.462009e+01,4.994095e+08,4.531685e+02,2.106821e-02
std,2.837119e+06,2.846723e+03,2.531508e+02,5.344721e+03,3.007688e+08,1.544648e-02,4.032163e+01,2.722388e+01,2.722388e+01,2.884926e+08,2.798574e+02,1.436118e-01
min,3.000000e+00,1.020000e+02,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
25%,2.628084e+06,2.103000e+03,1.900000e+02,1.000000e+03,0.000000e+00,1.000000e+00,1.600000e+01,9.880000e+00,9.880000e+00,2.495002e+08,2.220000e+02,0.000000e+00
50%,4.858011e+06,4.103000e+03,3.700000e+02,2.200000e+03,0.000000e+00,1.000000e+00,2.800000e+01,1.800000e+01,1.800000e+01,4.990000e+08,4.000000e+02,0.000000e+00
75%,7.393869e+06,7.102000e+03,5.800000e+02,4.100000e+03,3.426031e+08,1.000000e+00,4.800000e+01,3.100000e+01,3.100000e+01,7.494000e+08,6.800000e+02,0.000000e+00
max,9.999997e+06,9.909000e+03,9.930000e+02,9.990900e+04,9.999100e+08,9.000000e+01,6.017000e+03,6.017000e+03,6.017000e+03,9.999004e+08,9.990000e+02,1.000000e+00


In [4]:
# Locate the maximum values for the Original price after grouping by SKU 
trnsact.groupby('SKU')['Orgprice'].max().reset_index

<bound method Series.reset_index of SKU
3          30.00
4          49.00
5          50.00
8          80.00
20         19.99
           ...  
9999967    42.00
9999970    29.99
9999974    83.00
9999992    14.00
9999997    39.00
Name: Orgprice, Length: 714499, dtype: float64>

In [5]:
# We replace any values that have 0 Original price with the maximum value we found for each SKU 
max_org_price = trnsact.groupby('SKU')['Orgprice'].transform('max')
trnsact['Orgprice'] = trnsact['Orgprice'].mask(trnsact['Orgprice']<max_org_price, max_org_price)

In [6]:
trnsact['Orgprice'] = max_org_price


In [7]:
#Count unique values after you group by SKU
price_ranges_trn = trnsact.groupby('SKU')['Orgprice'].nunique()
price_ranges_trn

SKU
3          1
4          1
5          1
8          1
20         1
          ..
9999967    1
9999970    1
9999974    1
9999992    1
9999997    1
Name: Orgprice, Length: 714499, dtype: int64

In [8]:
# Here we check to see if there are any SKU values that have greater than 1 unique Orgprice after we grouped by SKU 
(price_ranges_trn > 1).value_counts()

False    714499
Name: Orgprice, dtype: int64

In [9]:
# Filter SKUs with more than 1 unique value
skus_with_multiple_values = price_ranges_trn[price_ranges_trn > 1].index.tolist()
skus_with_multiple_values

[]

In [10]:
# This is annother way for us to check by ordering in descending order whether there are any unique values greater than 1 
sorted_price_ranges = price_ranges_trn.sort_values(ascending=False)
sorted_price_ranges

SKU
3          1
6649025    1
6648944    1
6648945    1
6648947    1
          ..
3316729    1
3316741    1
3316776    1
3316778    1
9999997    1
Name: Orgprice, Length: 714499, dtype: int64

In [11]:
# We check whether there are any SKU's that still have 0 Original Price so we can determine what to do with them depending on the size of missing value
check_zero_value = (trnsact['Orgprice']==0).any()
check_zero_value

True

In [12]:
trnsact.tail(1000)

,SKU,Store,Register,Trannum,IntID,Saledate,Stype,Quantity,AMT,Orgprice,Duplicate,SEQ,MIC,Unknown
120915896,9999950,9103,841,3800,0,2004-09-25,P,1,13.5,14.0,13.50,563100167,281,0
120915897,9999950,9103,841,4200,0,2004-09-10,P,1,13.5,14.0,13.50,188000111,281,0
120915898,9999950,9103,841,4500,0,2005-03-05,P,1,13.5,14.0,13.50,631800178,281,0
120915899,9999950,9103,841,6800,986308581,2004-08-15,P,1,13.5,14.0,13.50,366000082,281,0
120915900,9999950,9103,841,8200,0,2004-08-19,R,1,13.5,14.0,13.50,552700083,281,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120916891,9999997,7507,750,2800,0,2005-04-23,P,1,39.0,39.0,39.00,593900116,395,0
120916892,9999997,7907,790,3200,0,2005-05-03,P,1,39.0,39.0,39.00,502600061,395,0
120916893,9999997,7907,800,700,0,2005-05-04,R,1,39.0,39.0,39.00,600900066,395,0
120916894,9999997,7907,800,700,0,2005-05-04,P,1,39.0,39.0,39.00,601000066,395,0


In [13]:
zero_price_prod = trnsact[trnsact['Orgprice']== 0]
zero_price_prod

,SKU,Store,Register,Trannum,IntID,Saledate,Stype,Quantity,AMT,Orgprice,Duplicate,SEQ,MIC,Unknown
154172,12790,4302,31,300,0,2005-06-10,P,1,0.0,0.0,0.0,170700087,281,0
154173,12790,4302,31,1000,0,2005-05-28,P,1,0.0,0.0,0.0,242800123,281,0
154174,12790,4302,31,2000,0,2005-03-30,P,1,0.0,0.0,0.0,216900066,281,0
154175,12790,4302,31,2200,0,2005-05-01,P,1,0.0,0.0,0.0,664800063,281,0
154176,12790,4302,31,2200,0,2005-05-21,P,1,0.0,0.0,0.0,467100112,281,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120181751,9937366,7907,960,400,0,2005-07-05,P,1,0.0,0.0,0.0,225500078,226,0
120181752,9937366,7907,960,1300,609309008,2005-06-05,P,1,0.0,0.0,0.0,617300058,226,0
120181753,9937366,8109,890,1200,0,2005-08-25,P,1,0.0,0.0,0.0,724900146,226,0
120181754,9937366,8109,890,2200,350006921,2005-07-06,P,1,0.0,0.0,0.0,412800080,226,0


In [14]:
zero_price_prod = trnsact[trnsact['Orgprice']== 0].count()

In [15]:
print(zero_price_prod)

SKU          12363
Store        12363
Register     12363
Trannum      12363
IntID        12363
Saledate     12363
Stype        12363
Quantity     12363
AMT          12363
Orgprice     12363
Duplicate    12363
SEQ          12363
MIC          12363
Unknown      12363
dtype: int64


In [16]:
# This is an example of the product with SKU: 12790 and we observe that all the samples have 0 original price 
SKU_missing = trnsact[trnsact['SKU'] == 12790]
SKU_missing

,SKU,Store,Register,Trannum,IntID,Saledate,Stype,Quantity,AMT,Orgprice,Duplicate,SEQ,MIC,Unknown
154172,12790,4302,31,300,0,2005-06-10,P,1,0.0,0.0,0.0,170700087,281,0
154173,12790,4302,31,1000,0,2005-05-28,P,1,0.0,0.0,0.0,242800123,281,0
154174,12790,4302,31,2000,0,2005-03-30,P,1,0.0,0.0,0.0,216900066,281,0
154175,12790,4302,31,2200,0,2005-05-01,P,1,0.0,0.0,0.0,664800063,281,0
154176,12790,4302,31,2200,0,2005-05-21,P,1,0.0,0.0,0.0,467100112,281,0
154177,12790,4302,31,2300,0,2005-06-20,P,1,0.0,0.0,0.0,863500060,281,0
154178,12790,4302,31,2600,0,2005-06-17,P,1,0.0,0.0,0.0,995300096,281,0
154179,12790,4302,31,3100,0,2005-06-17,P,1,0.0,0.0,0.0,995400096,281,0
154180,12790,4302,31,3200,0,2005-06-18,P,1,0.0,0.0,0.0,630400137,281,0
154181,12790,4302,31,3500,0,2005-05-14,P,1,0.0,0.0,0.0,515100111,281,0


### We identified 12363 unique SKU's that still have 0 original price so we decided to use the skstinfo table to deal with the remaining missing values

In [17]:
sksinfo = pd.read_csv('skstinfo.csv')
sksinfo.columns = ['SKU','Store','Cost','Retail','Unkown']
sksinfo

,SKU,Store,Cost,Retail,Unkown
0,3,103,123.36,440.00,0
1,3,104,123.36,440.00,0
2,3,202,123.36,440.00,0
3,3,203,123.36,440.00,0
4,3,204,123.36,440.00,0
...,...,...,...,...,...
39230140,9999997,2007,15.00,19.50,0
39230141,9999997,2707,15.00,9.75,0
39230142,9999997,3307,15.00,19.50,0
39230143,9999997,7507,15.00,19.50,0


In [18]:
# Group by unique_id and calculate the price range (non zero)
price_ranges = sksinfo.groupby('SKU')['Retail'].agg(lambda x: x.max() - x.min())
price_ranges

SKU
3           0.00
4           0.00
8           0.00
15          0.00
39          0.00
           ...  
9999973     0.00
9999974    41.75
9999991     0.00
9999992     0.00
9999997     9.75
Name: Retail, Length: 760212, dtype: float64

In [19]:
# Replace any 0 values in the Retail feature with NA    
sksinfo['Retail'].replace(0, np.nan, inplace=True)
# Convert the 'Retail' column to numeric, ignoring non-numeric values like NaN
sksinfo['Retail'] = pd.to_numeric(sksinfo['Retail'], errors='coerce')
# Group by SKU and calculate the average, excluding NaN values
average_prices = sksinfo.groupby('SKU')['Retail'].mean()

In [20]:
# Check that there are no zero mean values 
average_prices.sort_values(ascending=True)

SKU
662930     0.01
2079401    0.01
104241     0.01
9047798    0.01
224210     0.01
           ... 
9990425     NaN
9996713     NaN
9997101     NaN
9998448     NaN
9998945     NaN
Name: Retail, Length: 760212, dtype: float64

In [21]:
# These are the total NaN values that we replaced 
na_count = average_prices.isna().sum()
na_count

2761

## Create a new DataFrame

In [29]:
# Create a new dataframe 
sksinfo_reduced = pd.DataFrame({'SKU': average_prices.index, 'Avg_Retail_Price': average_prices.values})
sksinfo_reduced.head()

,SKU,Avg_Retail_Price
0,3,440.0
1,4,12.0
2,8,40.0
3,15,119.0
4,39,119.0


In [31]:
# Merge the two tables on 'SKU'
merged_table = pd.merge(trnsact, sksinfo_reduced[['SKU', 'Avg_Retail_Price']], on='SKU', how='left')

# Replace 0 values in 'original price' with corresponding 'retail price'
merged_table.loc[merged_table['Orgprice'] == 0, 'Orgprice'] = merged_table['Avg_Retail_Price']

# Drop the 'retail price' column if you no longer need it in the merged table
merged_table.drop(columns=['Avg_Retail_Price'], inplace=True)

# Print the result
print(merged_table)

               SKU  Store  Register  Trannum      IntID    Saledate Stype  \
0                3    202       290     1100  326708721  2005-01-18     P   
1                3    202       540     2700  326708721  2005-01-29     R   
2                3    303       500     2100   23702074  2004-08-18     P   
3                3    709       360      500          0  2005-08-14     P   
4                3    802       660      400          0  2005-08-09     P   
...            ...    ...       ...      ...        ...         ...   ...   
120916891  9999997   7507       750     2800          0  2005-04-23     P   
120916892  9999997   7907       790     3200          0  2005-05-03     P   
120916893  9999997   7907       800      700          0  2005-05-04     R   
120916894  9999997   7907       800      700          0  2005-05-04     P   
120916895  9999997   7907       800     4100  257808914  2005-07-31     P   

           Quantity    AMT  Orgprice  Duplicate        SEQ  MIC  Unknown  


In [33]:
check_zero_value_new = (merged_table['Orgprice']==0).any()
check_zero_value_new

False

In [35]:
merged_table['Orgprice'].sort_values(ascending=True)

27979775     0.01
95216330     0.01
95216331     0.01
95216332     0.01
95216333     0.01
             ... 
120181751     NaN
120181752     NaN
120181753     NaN
120181754     NaN
120745173     NaN
Name: Orgprice, Length: 120916896, dtype: float64

In [37]:
merged_table['SortedOrgprice'] = merged_table.groupby('SKU')['Orgprice'].transform(lambda x: x.sort_values(ascending=True))

## We observed that there were some Orgprice = 0.01 we explored the data and attemped to replace those with the mean value but we did not succeed

In [38]:
# Merge the two tables on 'SKU'
merged_table = pd.merge(trnsact, sksinfo_reduced[['SKU', 'Avg_Retail_Price']], on='SKU', how='left')

# Replace 0 values in 'original price' with corresponding 'retail price'
merged_table.loc[merged_table['Orgprice'] < 1, 'Orgprice'] = merged_table['Avg_Retail_Price']

# Drop the 'retail price' column if you no longer need it in the merged table
merged_table.drop(columns=['Avg_Retail_Price'], inplace=True)

# Print the result
print(merged_table)

               SKU  Store  Register  Trannum      IntID    Saledate Stype  \
0                3    202       290     1100  326708721  2005-01-18     P   
1                3    202       540     2700  326708721  2005-01-29     R   
2                3    303       500     2100   23702074  2004-08-18     P   
3                3    709       360      500          0  2005-08-14     P   
4                3    802       660      400          0  2005-08-09     P   
...            ...    ...       ...      ...        ...         ...   ...   
120916891  9999997   7507       750     2800          0  2005-04-23     P   
120916892  9999997   7907       790     3200          0  2005-05-03     P   
120916893  9999997   7907       800      700          0  2005-05-04     R   
120916894  9999997   7907       800      700          0  2005-05-04     P   
120916895  9999997   7907       800     4100  257808914  2005-07-31     P   

           Quantity    AMT  Orgprice  Duplicate        SEQ  MIC  Unknown  


In [39]:
merged_table['Orgprice'].sort_values(ascending=True)

109465012    0.01
109464867    0.01
109464868    0.01
109464869    0.01
109464870    0.01
             ... 
120210088     NaN
120391981     NaN
120391982     NaN
120467100     NaN
120745173     NaN
Name: Orgprice, Length: 120916896, dtype: float64

In [42]:
# These are the total NaN values in the merged table
na_count_merged = merged_table.isna().sum()
na_count_merged

SKU              0
Store            0
Register         0
Trannum          0
IntID            0
Saledate         0
Stype            0
Quantity         0
AMT              0
Orgprice     56620
Duplicate        0
SEQ              0
MIC              0
Unknown          0
dtype: int64

In [43]:
# These are the total NaN values in the original table there were none so we discovered that when we merged the tables we assigned the NaN as well. 
na_count_trnsact = trnsact.isna().sum()
na_count_trnsact

SKU          0
Store        0
Register     0
Trannum      0
IntID        0
Saledate     0
Stype        0
Quantity     0
AMT          0
Orgprice     0
Duplicate    0
SEQ          0
MIC          0
Unknown      0
dtype: int64

In [45]:
# We want to replace NaN into 0 
numeric_value = 0
merged_table['Orgprice'].fillna(numeric_value, inplace=True)

## EDA on Stype and Other Features 

#### 1) The percentage of the returned items as opposed to purchase 

In [49]:
total_returns = (merged_table['Stype'] == 'R').sum()
total_returns

9267802

In [65]:
# Count the occurrences of each category
stype_counts = merged_table['Stype'].value_counts()

# Calculate total returns and total purchases
total_returns = stype_counts.get('R', 0)
total_purchases = stype_counts.get('P', 0)

# Calculate the total percentage of returns and purchases
total_samples = len(merged_table)
percent_returns = (total_returns / total_samples) * 100
percent_purchases = (total_purchases / total_samples) * 100

# Print the results
print(f'Total Returns: {total_returns}')
print(f'Total Purchases: {total_purchases}')
print(f'Percentage of Returns: {percent_returns:.2f}%')
print(f'Percentage of Purchases: {percent_purchases:.2f}%')

Total Returns: 9267802
Total Purchases: 111649094
Percentage of Returns: 7.66%
Percentage of Purchases: 92.34%
